In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import nltk
from sklearn.model_selection import GridSearchCV
nltk.download('wordnet')
from sklearn.svm import SVC

[nltk_data] Downloading package wordnet to /Users/nathan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
col_names = ['marketplace','customer_id','review_id','product_id','product_parent','product_title','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_body','review_date']
cols = {}
for i in range(len(col_names)):
    print (str(i)+': '+col_names[i])
    cols[col_names[i]] = i 

0: marketplace
1: customer_id
2: review_id
3: product_id
4: product_parent
5: product_title
6: product_category
7: star_rating
8: helpful_votes
9: total_votes
10: vine
11: verified_purchase
12: review_headline
13: review_body
14: review_date


In [3]:
np.random.seed(500)

In [4]:
df = pd.read_csv('data/sample_02.csv')

df = df.sample(frac=.05, random_state=1)

In [5]:
helpful_percentage = []
for pos, total in zip(df['8'],df['9']):
    if total>0:
        helpful_percentage.append(float(pos)/float(total))
    else:
        helpful_percentage.append(0)
df['15'] = helpful_percentage

In [6]:
print(len(df))
df.head(3)

3123


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
53304,US,49786746,R217S4F2FV9SRT,0965824233,173013649,The NAET Guide Book (4th Ed.),Books,5,13,29,N,N,The best Guide Book for a patient,It's a very good guide-book where you can get ...,11050,0.448276
21985,US,52661944,R295CEHQLEXS5G,0515120871,434295698,Finding the Dream (Dream Trilogy),Books,3,1,3,N,N,Sloppy writing,"This book, like all of Nora Roberts' books, is...",12200,0.333333
58996,US,52651244,RNAI0W4D8WWWL,0821761455,199563422,With All My Heart,Books,3,2,2,N,N,Not as good as the other two in the trilogy,I had been waiting patiently (sometimes unpati...,10650,1.000000


In [7]:
Corpus = pd.DataFrame()
Corpus['text'] = df['13']
lst = []
for x in df['15']:
    if x>=.66:
        lst.append('good')
    else:
        lst.append('bad')
Corpus['label'] = lst

In [8]:
len(Corpus['label'])

3123

In [9]:
# Step - a : Remove blank rows if any.
Corpus['text'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['text'] = [entry.lower() for entry in Corpus['text']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['text']= [word_tokenize(entry) for entry in Corpus['text']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
lst = []
for index,entry in enumerate(Corpus['text']):
    
    if index%1000 ==0:
        print(index)
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    
    #Corpus.loc[index,'text_final'] = str(Final_words)
    lst.append(str(Final_words))
    
Corpus['text_final'] = lst

0
1000
2000
3000


In [10]:
len(lst)

3123

In [11]:
Corpus.to_csv('data/corpus.csv')


In [12]:
Corpus = pd.read_csv('data/corpus.csv')

In [13]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.3)




In [14]:
Train_Y

2613    good
635     good
653     good
534     good
2310    good
        ... 
287      bad
3005    good
273     good
951     good
2906     bad
Name: label, Length: 2186, dtype: object

In [15]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [16]:
Tfidf_vect = TfidfVectorizer(max_features=10000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [17]:
print(Tfidf_vect.vocabulary_)

{'good': 3854, 'get': 3800, 'quick': 7209, 'answer': 326, 'many': 5461, 'question': 7205, 'book': 1039, 'like': 5172, 'nora': 6197, 'robert': 7625, 'enjoyable': 3120, 'character': 1455, 'interesting': 4529, 'relationship': 7424, 'michael': 5676, 'laura': 5035, 'daughter': 2212, 'really': 7315, 'lovely': 5300, 'however': 4226, 'writing': 9947, 'kinda': 4901, 'sloppy': 8266, 'quot': 7224, 'dare': 2192, 'dream': 2797, 'everyone': 3245, 'surprise': 8803, 'josh': 4768, 'fell': 3467, 'love': 5296, 'seem': 7941, 'know': 4937, 'go': 3843, 'happen': 4002, 'explanation': 3332, 'find': 3515, 'coin': 1631, 'cliff': 1586, 'treasure': 9314, 'hide': 4127, 'cave': 1405, 'two': 9418, 'horse': 4210, 'name': 6003, 'tell': 8982, 'twice': 9413, 'byron': 1272, 'kate': 4841, 'expect': 3314, 'maybe': 5540, 'picky': 6816, 'point': 6891, 'confuse': 1795, 'follow': 3586, 'plot': 6877, 'inconsistency': 4385, 'detail': 2462, 'think': 9090, 'bear': 781, 'series': 8006, 'much': 5933, 'well': 9799, 'write': 9944, 'wa

In [18]:
print(Train_X_Tfidf[3])

  (0, 9931)	0.017727968726026822
  (0, 9921)	0.022636458293911976
  (0, 9913)	0.020622289561168603
  (0, 9897)	0.11631682876128797
  (0, 9875)	0.0509815613249413
  (0, 9845)	0.03576602168158849
  (0, 9799)	0.01815821773044599
  (0, 9765)	0.04087407184589167
  (0, 9734)	0.020651211052824064
  (0, 9671)	0.04546611666770209
  (0, 9643)	0.04107017751441287
  (0, 9568)	0.04061270806836266
  (0, 9567)	0.040182164606193296
  (0, 9562)	0.020174447916326714
  (0, 9535)	0.05442063797603633
  (0, 9509)	0.056053407872725565
  (0, 9418)	0.0228485638076365
  (0, 9399)	0.02742863534042575
  (0, 9382)	0.04887489086990356
  (0, 9266)	0.05815841438064399
  (0, 9235)	0.03687493166375501
  (0, 9110)	0.0878398820216019
  (0, 8958)	0.07574205637901381
  (0, 8908)	0.020039938101660278
  (0, 8805)	0.08715909621645702
  :	:
  (0, 1576)	0.03003847151062476
  (0, 1460)	0.04264417498377856
  (0, 1455)	0.041244579122337206
  (0, 1445)	0.05584082071084154
  (0, 1380)	0.029934680722703763
  (0, 1336)	0.0493487914282

In [19]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  61.792956243329776


In [20]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=6, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  62.4332977588047


In [21]:
sum(Corpus['label']=='good')/len(Corpus)

0.6407300672430355

In [22]:
sum(Corpus['label']=='good')

2001

## Grid Search SVM

In [23]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1,2,3,4 ,10, 100, 1000]}]

scores = ['precision', 'recall']


In [24]:
clf = GridSearchCV(SVC(), tuned_parameters, cv=5,scoring='%s_macro' % scores[0])
clf.fit(Train_X_Tfidf,Train_Y)

/Users/nathan/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']},
                         {'C': [1, 2, 3, 4, 10, 100, 1000],
                          'kernel': ['linear']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='precision_macro', verbose=0)

In [25]:
print(clf.best_params_)

{'C': 1, 'kernel': 'linear'}
